# Team Assignment - Intentionally Blank

The following libraries should be installed before proceeding further:
- pandas
- numpy

In [293]:
import pandas as pd
import numpy as np
from datetime import datetime

First of all the important data files must be read for further use.

In [294]:
# Reading weather data and data from philadelphia_2017
data_weather = pd.read_csv("weather_hourly_philadelphia.csv")
data_philadelphia_2017 = pd.read_csv("philadelphia_2017.csv")

In [295]:
df_philadelphia_2017 = pd.DataFrame(data_philadelphia_2017)
df_philadelphia_2017.loc[:,"start_time"] = pd.to_datetime(df_philadelphia_2017["start_time"])
df_philadelphia_2017.loc[:,"end_time"] = pd.to_datetime(df_philadelphia_2017["end_time"])
df_philadelphia_2017.sort_values(["start_time"], inplace = True)
df_philadelphia_2017.reset_index(drop=True, inplace = True)
df_philadelphia_2017


,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
0,2017-01-01 00:05:00,2017-01-01 00:16:00,3046,3041,5347,Indego30,2nd & Market,"Girard Station, MFL"
1,2017-01-01 00:21:00,2017-01-01 00:57:00,3110,3054,3364,Walk-up,Del. River Trail & Penn St.,Rodin Museum
2,2017-01-01 00:22:00,2017-01-01 00:57:00,3110,3054,2536,Walk-up,Del. River Trail & Penn St.,Rodin Museum
3,2017-01-01 00:27:00,2017-01-01 00:39:00,3041,3005,5176,Indego30,"Girard Station, MFL","Welcome Park, NPS"
4,2017-01-01 00:28:00,2017-01-01 00:36:00,3047,3124,5370,Walk-up,"Independence Mall, NPS",Race Street Pier
...,...,...,...,...,...,...,...,...
788902,2017-12-31 23:05:00,2017-12-31 23:33:00,3070,3124,3708,Indego30,"Spring Garden Station, MFL",Race Street Pier
788903,2017-12-31 23:11:00,2018-01-01 11:03:00,3107,3165,5117,Indego30,33rd & Reservoir,24th & Race SRT
788904,2017-12-31 23:18:00,2017-12-31 23:25:00,3033,3046,11933,Indego30,10th & Chestnut,2nd & Market
788905,2017-12-31 23:39:00,2017-12-31 23:40:00,3163,3163,6725,Indego30,25th & Locust,25th & Locust


In [300]:
df_weather = pd.DataFrame(data_weather)
df_weather.loc[:, "date_time"] = pd.to_datetime(df_weather["date_time"])
df_weather.sort_values(["date_time"], inplace = True)
df_weather.reset_index(drop=True, inplace =True)
df_weather

,date_time,max_temp,min_temp,precip
0,2015-01-01 06:00:00,-1.7,-1.7,0.0
1,2015-01-01 07:00:00,-1.7,-1.7,0.0
2,2015-01-01 08:00:00,-1.7,-1.7,0.0
3,2015-01-01 09:00:00,-1.1,-1.1,0.0
4,2015-01-01 10:00:00,-1.1,-1.1,0.0
...,...,...,...,...
43843,NaT,NaN,NaN,NaN
43844,NaT,NaN,NaN,NaN
43845,NaT,NaN,NaN,NaN
43846,NaT,NaN,NaN,NaN


In [301]:
df_weather_2017 = df_weather[(df_weather["date_time"]>= "2017-01-01 00:00:00") & (df_weather["date_time"]< "2018-01-01 00:00:00" )]
df_weather_2017.reset_index(drop=True, inplace = True)
df_weather_2017

,date_time,max_temp,min_temp,precip
0,2017-01-01 00:00:00,6.1,6.1,0.0
1,2017-01-01 01:00:00,6.1,6.1,0.0
2,2017-01-01 02:00:00,6.7,6.7,0.0
3,2017-01-01 03:00:00,7.2,7.2,0.0
4,2017-01-01 04:00:00,7.8,7.8,0.0
...,...,...,...,...
8750,2017-12-31 19:00:00,-8.9,-8.9,0.0
8751,2017-12-31 20:00:00,-8.9,-8.9,0.0
8752,2017-12-31 21:00:00,-9.4,-9.4,0.0
8753,2017-12-31 22:00:00,-10.0,-10.0,0.0


In [302]:
df_weather_2017[df_weather_2017.all(axis='columns').isnull() == True]

,date_time,max_temp,min_temp,precip


In [303]:
df_philadelphia_2017[df_philadelphia_2017.all(axis='columns').isnull() == True]

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name


In [304]:
df_philadelphia_2017[df_philadelphia_2017.duplicated()]

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name
621519,2017-10-06 07:02:00,2017-10-06 07:16:00,3120,3111,11905,Indego30,31st & Girard,"Parkside & Belmont, Case Building"
622143,2017-10-06 09:18:00,2017-10-06 09:38:00,3123,3111,5185,Indego30,22nd & Cecil B. Moore,"Parkside & Belmont, Case Building"
622166,2017-10-06 09:24:00,2017-10-06 09:56:00,3111,3004,11905,Indego30,"Parkside & Belmont, Case Building",Municipal Services Building Plaza
622390,2017-10-06 10:58:00,2017-10-06 11:18:00,3095,3111,5385,Indego30,29th & Diamond,"Parkside & Belmont, Case Building"
622432,2017-10-06 11:16:00,2017-10-06 11:46:00,3111,3107,5185,Indego30,"Parkside & Belmont, Case Building",33rd & Reservoir
622582,2017-10-06 12:09:00,2017-10-06 12:24:00,3120,3111,5284,Indego30,31st & Girard,"Parkside & Belmont, Case Building"
623181,2017-10-06 15:38:00,2017-10-06 16:30:00,3111,3040,5284,Walk-up,"Parkside & Belmont, Case Building",17th & Spring Garden
623186,2017-10-06 15:39:00,2017-10-06 16:29:00,3111,3040,2650,Walk-up,"Parkside & Belmont, Case Building",17th & Spring Garden
623578,2017-10-06 17:09:00,2017-10-06 17:36:00,3078,3111,11007,Indego30,19th & Market,"Parkside & Belmont, Case Building"
624259,2017-10-06 19:29:00,2017-10-06 19:53:00,3111,3096,11007,Indego30,"Parkside & Belmont, Case Building",29th & Dauphin


In [305]:
df_philadelphia_2017.drop_duplicates(inplace= True)
df_philadelphia_2017[df_philadelphia_2017.duplicated()]

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name


In [306]:
df_weather_2017[df_weather_2017.duplicated()]

,date_time,max_temp,min_temp,precip
59,2017-01-03 10:00:00,5.0,5.0,0.0
160,2017-01-07 15:00:00,-6.1,-6.1,1.0
334,2017-01-14 21:00:00,1.7,1.7,0.0
340,2017-01-15 03:00:00,1.1,1.1,0.0
342,2017-01-15 05:00:00,1.1,1.1,0.0
...,...,...,...,...
7811,2017-11-22 14:00:00,11.7,11.7,0.0
8138,2017-12-06 06:00:00,13.9,13.9,0.0
8227,2017-12-09 23:00:00,-1.1,-1.1,1.0
8429,2017-12-18 09:00:00,3.9,3.9,0.0


In [307]:
df_weather_2017 = df_weather_2017.drop_duplicates()
df_weather_2017[df_weather_2017.duplicated()]

,date_time,max_temp,min_temp,precip


In [308]:
df_philadelphia_2017.loc[:,"duration"] = df_philadelphia_2017["end_time"] - df_philadelphia_2017["start_time"] 
df_philadelphia_2017.loc[:,"weekday"] = df_philadelphia_2017["start_time"].dt.weekday
df_philadelphia_2017.loc[:,"month"] = df_philadelphia_2017["start_time"].dt.month
df_philadelphia_2017.loc[:,"hour"] = df_philadelphia_2017["start_time"].dt.hour
df_philadelphia_2017

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_station_name,end_station_name,duration,weekday,month,hour
0,2017-01-01 00:05:00,2017-01-01 00:16:00,3046,3041,5347,Indego30,2nd & Market,"Girard Station, MFL",0 days 00:11:00,6,1,0
1,2017-01-01 00:21:00,2017-01-01 00:57:00,3110,3054,3364,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:36:00,6,1,0
2,2017-01-01 00:22:00,2017-01-01 00:57:00,3110,3054,2536,Walk-up,Del. River Trail & Penn St.,Rodin Museum,0 days 00:35:00,6,1,0
3,2017-01-01 00:27:00,2017-01-01 00:39:00,3041,3005,5176,Indego30,"Girard Station, MFL","Welcome Park, NPS",0 days 00:12:00,6,1,0
4,2017-01-01 00:28:00,2017-01-01 00:36:00,3047,3124,5370,Walk-up,"Independence Mall, NPS",Race Street Pier,0 days 00:08:00,6,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
788902,2017-12-31 23:05:00,2017-12-31 23:33:00,3070,3124,3708,Indego30,"Spring Garden Station, MFL",Race Street Pier,0 days 00:28:00,6,12,23
788903,2017-12-31 23:11:00,2018-01-01 11:03:00,3107,3165,5117,Indego30,33rd & Reservoir,24th & Race SRT,0 days 11:52:00,6,12,23
788904,2017-12-31 23:18:00,2017-12-31 23:25:00,3033,3046,11933,Indego30,10th & Chestnut,2nd & Market,0 days 00:07:00,6,12,23
788905,2017-12-31 23:39:00,2017-12-31 23:40:00,3163,3163,6725,Indego30,25th & Locust,25th & Locust,0 days 00:01:00,6,12,23
